# The Goal
Our general goal is to identify standing dead trees among a dataset of aerial view forests photos. 

# The Dataset

# Deep Learning Approach #1 - Unet + VGG19 for segmentation and classification.
We found a downstream task similar to our goal where a team of reasearchers had a series of MRI lung scans and were tasked with classifing the afflicted regions of different lung diseases including COVID-19 usin segmented regions and labelling those regions with which class of affliction. Here is the paper where they did this https://pmc.ncbi.nlm.nih.gov/articles/PMC9497601/. 

### Import Required Packages and Global Variables

In [ ]:
# Import all necessary packages
import os, time
import numpy as np
import cv2
from PIL import Image
from glob import glob
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import segmentation_models_pytorch as smp
from tqdm import tqdm

# Global Constraints
# Dimensions of the image inputs into Unet after resizing
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 8
# Configurable in case of hardware limitations
EPOCHS = 25
LR = 0.001
# Paths to the data directories
RBD_DIR = 'USA_segmentation/RGB_images'
NIR_DIR = 'USA_segmentation/NIR_images'
MASK_DIR = 'USA_segmentation/masks'
# Test proportion of full dataset
TEST_SIZE = 0.2
RANDOM_SEED = 42

### Retrieve and Split Labelled Data
The data from our project directory has to be read and split in order to be oprocess and used for training. 

In [ ]:
# Read in file paths from given directories and splits them into train and test sets
# Args:
#   rgb_dir (str): Path from current file directory containing the RGB photos
#   nir_dir (str): Path from current file directory containing the NIR photos
#   mask_dir (str): Path from current file directory containing the masks
#   test_size (float): Proportion of entire dataset contributed to tests
#   random_state (int): Random seed
# Returns:
#   tuple: A tuple containin two lists (train_data, test_data)
def read_and_split_data(rgb_dir, nir_dir, mask_dir, test_size=TEST_SIZE, random_state=RANDOM_SEED):
    # Get all RGB images sorted 
    rgb_files = sorted(glob(os.path.join(rgb_dir, "RGB_*.png")))
    
    tags = [os.path.basename(f).replace("RGB_", "").replace(".png", "") for f in rgb_files]
    
    pairs = []
    # Iterate though each identifying tag to find corresponding RGB, NIR, and masks
    for tag in tags:
        rgb_path = os.path.join(rgb_dir, f"RGB_{tag}.png")
        nir_path = os.path.join(rgb_dir, f"NIR_{tag}.png")
        mask_path = os.path.join(rgb_dir, f"mask_{tag}.png")
        
        # Check if all three corresponding files exist
        if os.path.exists(rgb_path) and os.path.exists(nir_path) and os.path.exists(mask_path):
            pairs.append({
                'rgb': rgb_path,
                'nir': nir_path,
                'mask': mask_path,
                'tag': tag
            })
        else:
            print(f"Skipping the {tag} file due to a missing file.")
            
    if not pairs:
        raise ValueError("No complete data pairs were found")
    
    # Split the collected data pairs into training and testing sets
    train_data, test_data = train_test_split(pairs, test_size=test_size, random_state=random_state)
    print(f"Total samples found: {len(pairs)}")
    print(f"Train samples: {len(train_data)}")
    print(f"Test samples: {len(test_data)}")
    
    return train_data, test_data
    

### Image Preprocessing

In [ ]:
# Initializes a custom dataset class for loading and preprocessing the forest images
# Args:
#   data_pairs (list): list of dicts with each entry having paths to rgb, nir, and mask
#   image_size (tuple): The target (width, height) for resizing images
class ForestDataset(Dataset):
    def __init__(self, data_pairs, image_size, transform=None):
        self.data_pairs = data_pairs
        self.image_size = image_size
        self.transform = transform

    # Returns the total number of samples in the dataset
    def __len__(self):
        return len(self.data_pairs)

    # Loads, preprocesses, and returns a single sample (input image and mask)
    # Args:
    #     idx (int): Index of the sample to retrieve
    # Returns:
    #     tuple: A tuple containing the preprocessed input tensor (4 channels)
    #            and the mask tensor (1 channel)
    def __getitem__(self, idx):
        item = self.data_pairs[idx]
        rgb_path = item['rgb']
        nir_path = item['nir']
        mask_path = item['mask']

        # Load images using PIL
        # RGB image is converted to 'RGB'
        # NIR and mask images are converted to 'L' (grayscale)
        rgb_image = Image.open(rgb_path).convert("RGB")
        nir_image = Image.open(nir_path).convert("L")
        mask_image = Image.open(mask_path).convert("L")

        # Resize images to the specified IMAGE_SIZE
        rgb_image = rgb_image.resize(self.image_size)
        nir_image = nir_image.resize(self.image_size)
        mask_image = mask_image.resize(self.image_size)

        # Convert PIL images to NumPy arrays and normalize pixel values to [0, 1]
        # RGB: (H, W, 3)
        # NIR: (H, W)
        # Mask: (H, W)
        rgb_np = np.array(rgb_image).astype(np.float32) / 255.0
        nir_np = np.array(nir_image).astype(np.float32) / 255.0
        mask_np = np.array(mask_image).astype(np.float32) / 255.0

        # Ensure the mask is strictly binary (0 or 1)
        mask_np[mask_np > 0.5] = 1.0
        mask_np[mask_np <= 0.5] = 0.0

        # Convert NumPy arrays to PyTorch tensors.
        # PyTorch expects image tensors in (C, H, W) format
        # RGB (H, W, 3) -> (3, H, W)
        rgb_tensor = torch.from_numpy(rgb_np).permute(2, 0, 1)
        # NIR (H, W) -> (1, H, W) by adding a channel dimension
        nir_tensor = torch.from_numpy(nir_np).unsqueeze(0)

        # Concatenate RGB and NIR tensors along the channel dimension to create a 4-channel input
        input_tensor = torch.cat((rgb_tensor, nir_tensor), dim=0)

        # Mask tensor also needs a channel dimension (1, H, W) for the loss function
        mask_tensor = torch.from_numpy(mask_np).unsqueeze(0)

        # Apply additional transforms if provided 
        if self.transform:
            input_tensor = self.transform(input_tensor)
            # Apply transform to mask if it's a spatial transform like rotation/flip
            mask_tensor = self.transform(mask_tensor)

        return input_tensor, mask_tensor

###  Define and Train Unet Segmentation Model 

In [ ]:
# Trains a U-Net segmentation model using the provided training data
# Args:
#     train_data (list): List of dictionaries containing training image paths
#     num_epochs (int): Number of epochs to train the model
#     batch_size (int): Number of samples per batch
#     learning_rate (float): Learning rate for the optimizer
#     image_size (tuple): Target image size (width, height)
# Returns:
#     tuple: A tuple containing the trained model and the total training time in seconds
def train_unet_model(train_data, num_epochs=EPOCHS, batch_size=BATCH_SIZE, learning_rate=LR, image_size=IMAGE_SIZE):

    # Determine the device to use (GPU or CPU).
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device for training: {device}")

    # Initialize the training dataset and data loader
    # Data transformation can be added here if more augmentations are desired
    train_dataset = ForestDataset(train_data, image_size, transform=None)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count() // 2 or 1)

    # Initialize the U-Net model from segmentation_models_pytorch
    # 'resnet34' is chosen as the encoder, pre-trained on 'imagenet' for transfer learning
    # in_channels=4 because our input combines 3 RGB channels and 1 NIR channel
    # classes=1 for binary segmentation (dead trees vs. background)
    # activation=None because BCEWithLogitsLoss handles the sigmoid activation internally
    model = smp.Unet(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=4,
        classes=1,
        activation=None
    )
    model.to(device) # Move model to the selected device (GPU/CPU)

    # Define the loss function and optimizer
    # BCEWithLogitsLoss is numerically stable and recommended for binary segmentation
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    start_time = time.time() 

    # Training loop
    for epoch in range(num_epochs):
        model.train() 
        running_loss = 0.0
        # Iterate over batches from the training data loader
        for inputs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs = inputs.to(device) 
            masks = masks.to(device)   

            optimizer.zero_grad() 
            outputs = model(inputs) 
            loss = loss_function(outputs, masks) 
            loss.backward() 
            optimizer.step() 

            running_loss += loss.item() * inputs.size(0) 

        epoch_loss = running_loss / len(train_dataset) 
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    end_time = time.time() 
    training_time = end_time - start_time
    print(f"Training finished in {training_time:.2f} seconds.")

    return model, training_time

### Fine Tuning/Training

# Evaluation

In [ ]:
# Evaluates the trained segmentation model on the test dataset.
# Args:
#     model (torch.nn.Module): The trained U-Net model.
#     test_data (list): List of dictionaries containing test image paths.
#     image_size (tuple): Target image size (width, height).
#     batch_size (int): Number of samples per batch.
# Returns:
#     tuple: A tuple containing the mean IoU score and the total testing time in seconds.
def evaluate_model(model, test_data, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device for evaluation: {device}")

    # Initialize the test dataset and data loader
    test_dataset = ForestDataset(test_data, image_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count() // 2 or 1)

    model.eval() 
    all_preds = [] 
    all_masks = [] 

    print("Starting model evaluation...")
    start_time = time.time() 

    # Disable gradient calculation during inference to save memory and speed up computation
    with torch.no_grad():
        for inputs, masks in tqdm(test_loader, desc="Evaluating"):
            inputs = inputs.to(device)
            masks = masks.to(device)  

            outputs = model(inputs) 
            # Apply sigmoid to outputs to get probabilities, then threshold at 0.5 for binary predictions
            preds = torch.sigmoid(outputs)
            preds = (preds > 0.5).float()
            
            # Move predictions and masks to CPU and convert to NumPy
            all_preds.append(preds.cpu().numpy()) 
            all_masks.append(masks.cpu().numpy()) 

    end_time = time.time() 
    testing_time = end_time - start_time
    print(f"Evaluation finished in {testing_time:.2f} seconds.")

    # Concatenate all predictions and masks into single NumPy arrays
    all_preds = np.vstack(all_preds)
    all_masks = np.vstack(all_masks)

    # Calculate IoU (Jaccard Similarity Coefficient)
    # Small smoothing value to avoid division by zero
    epsilon = 1e-6 

    intersection = (all_preds * all_masks).sum(axis=(1, 2, 3)) # Sum over H, W, C for each image in batch
    union = (all_preds + all_masks).sum(axis=(1, 2, 3)) - intersection
    iou_scores = (intersection + epsilon) / (union + epsilon) # Calculate IoU for each image
    
    # Calculate the mean IoU across all test samples.
    mean_iou = np.mean(iou_scores) 
    print(f"Mean IoU (Jaccard Similarity Coefficient): {mean_iou:.4f}")

    return mean_iou, testing_time

# Prediction Post Processing